Import important libraries first!

In [22]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Getting the xml from Wikipedia and processing it using the BeautifulSoup Library

In [23]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

Extracting the table element from the Wikipedia page and creating the dictionary to create the DataFrame later.

In [24]:
table = soup.find('table').tbody
table_rows = table.find_all('tr')
data_dict = {'PostalCode':[],'Borough':[],'Neighborhood':[]}


Processing the rows so that the Borough that are not assigned are ignored and the neighborhoods that are not assigned copy from the bourough.
Then for each row the processed information is inserted to the dictionary

In [25]:
for row in table_rows:
    elements = row.find_all('td')
    if len(elements)==0:
        continue
    if elements[1].string == "Not assigned" and elements[2].string=="Not assigned\n":
        continue
    elif elements[1].text != 'Not assigned' and elements[2].text=='Not assigned\n':
        elements[2].string = elements[1].text
    data_dict['PostalCode'].append(elements[0].string)
    data_dict['Borough'].append(elements[1].string)
    data_dict['Neighborhood'].append(elements[2].text.strip()+",")    

The final dictionary is turned into a pandas DataFrame then the groupby method is called so that duplicate postalcodes are consolidated.
Then sun concatenates the neighborhoods giving the final dataframe

In [28]:
data_df = pd.DataFrame.from_dict(data_dict)
data_df = data_df.groupby(["PostalCode","Borough"],as_index=False).sum()

This gives the shape of the final DataFrame and the head of it

In [30]:
print(data_df.shape)
data_df.head(10)

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern,"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union,"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill,"
3,M1G,Scarborough,"Woburn,"
4,M1H,Scarborough,"Cedarbrae,"
5,M1J,Scarborough,"Scarborough Village,"
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park,"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge,"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West,"
9,M1N,Scarborough,"Birch Cliff,Cliffside West,"
